<a href="https://colab.research.google.com/github/hamimmahmud0/animpahe_downloader/blob/master/Animepahe_Downloader_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Form
Fill up the form:

In [ ]:
from google.colab import drive
drive.mount('/drive')

#@markdown ---

URL = 'https://animepahe.com/anime/453da1c1-e3fe-7f1b-7655-4ee278f5692d' #@param {type: "string"}
ANIME_NAME = 'Clannad' #@param {type: "string"}
DOWNLOAD_DIR = '/Animpahe' #@param {type: "string"}
#@markdown Keep quality 0 to download highest available quality.
QUALITY = 0 #@param {type: "number"}

#@markdown ---
DOWNLOAD_DIR = '/drive/My Drive' + DOWNLOAD_DIR #save to drive
print('Your video will be saved at ' + DOWNLOAD_DIR)
print('Go to Drive: https://drive.google.com/drive/my-drive')

u_time = 0
u_t = []

# Program code

In [ ]:
!pip3 install http.client
!pip3 install brotli
!sudo apt-get install ffmpeg

In [ ]:
import subprocess
import os
from http.client import HTTPSConnection
import json
import threading
from google.colab import output
from google.colab import files
import time
import socket, ssl, time, gzip

In [ ]:
def list_parser(id,page):
  conn = HTTPSConnection('animepahe.com')
  conn.request('GET','https://animepahe.com/api?m=release&id='+id+'&l=30&sort=episode_desc&page='+page)
  episode_json = conn.getresponse().read().decode("utf-8")
  return json.loads(episode_json)  

def fetch_session(url):
  conn = HTTPSConnection('animepahe.com')
  anime = url.split('/')[4]
  conn.request('GET','/anime/'+anime)
  response = conn.getresponse()
  page_body = response.read().decode("utf-8")
  req_index = page_body.find("$.getJSON('/api?") - 1
  id = page_body[req_index:].split('\n')[0].split('&')[1].split('=')[1]
  conn.request('GET','https://animepahe.com/api?m=release&id='+id+'&l=30&sort=episode_desc&page=1')
  episode_json = conn.getresponse().read().decode("utf-8")
  episode_json = json.loads(episode_json)  
  sessions = []
  if 'last_page' in episode_json:
    pages = episode_json['last_page']
    x = 1
    while x <= pages:
      episode_json = list_parser(str(id),str(x))
      for episode in episode_json['data']:
        sessions.append([str(episode["episode"]),episode["session"]])
      x=x+1
  else:
    for episode in episode_json['data']:
      sessions.append([str(episode["episode"]),episode["session"]])
  return (id, sessions[::-1])

In [ ]:
def fetch_kwik_data(id,data):
  kwik_data = []
  keys = []
  data_length = len(data)
  i = 0
  for x in data:
    i = i+1
    epi = x[0]
    video_info = fetch_video_data_as_json(id,x[1])['data']
    for x in video_info:
      for key in x.keys():
        if not key in keys:
          keys.append(key)
        temp = [str(key),epi,x[key]["disc"],x[key]["kwik"]]
        #print('Episode: '+epi+' Quality: '+temp[0]+'p Disc: '+temp[2]+' URL: '+temp[3])        
        kwik_data.append(temp)
    show_pogress(i/data_length)
  return kwik_data, keys
      
def get_highest_quality(qualities):
  highest = 0
  for x in qualities:
    if int(x) > highest:
      highest = int(x)
  return str(highest)

In [ ]:
def extract_data_from_response(response):
  parts = response.split(b'\r\n\r\n')
  head = parts[0]
  body = parts[1]
  body = body.split(b'\r\n')
  i = 0
  refined_body = b''
  while len(body) > i:
    if i%2 == 1:
      refined_body = refined_body + body[i]
    i = i + 1
  return (head, refined_body)

def fetch_nextstream_url(URL,animepahe='https://animepahe.com/play/852a64f0-c84a-3981-0321-9d0716ab8dd5/dc27f599c18cd7c06687f6c948e530f739f51b0f3822e8c1f4b96292a9357381'):
  import socket, ssl, brotli
  HOST = URL.split('/')[2]
  PATH = '/e/'+URL.split('/')[4]
  HEADER = """host: kwik.cx
:authority: kwik.cx
:method: GET
:path: """+PATH+"""
:scheme: https
accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
accept-encoding: gzip, deflate, br
accept-language: en-US,en;q=0.9,bn;q=0.8
referer: """+animepahe+"""
sec-fetch-dest: iframe
sec-fetch-mode: navigate
sec-fetch-site: cross-site
sec-fetch-user: ?1
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36
"""

  HEADER.replace('\n','\r\n')
  context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
  s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  s_sock = context.wrap_socket(s, server_hostname=HOST)
  s_sock.connect((HOST,443))
  s_sock.send(("GET "+PATH+" HTTP/1.1\r\n"+HEADER+"\r\n").encode('utf-8'))
  response = b''
  while True:
    data = s_sock.recv(4096)
    response = response + data
    if data == b'0\r\n\r\n':
      break
  s_sock.close()
  parts = response.split(b'\r\n\r\n')
  #head = parts[0]
  body = parts[1]
  body = body.split(b'\r\n')
  i = 0
  refined_body = b''
  while len(body) > i:
    if i%2 == 1:
      refined_body = refined_body + body[i]
    i = i + 1
  refined_body = brotli.decompress(refined_body).decode()
  refined_body = refined_body[refined_body.find('<script')+1:]
  refined_body = refined_body[refined_body.find('<script')+1:]
  refined_body = refined_body[refined_body.find('<script')+1:]
  refined_body = refined_body[refined_body.find('<script')+1:]
  refined_body = refined_body[refined_body.find('<script')+1:]
  refined_body = refined_body[refined_body.find('<script')+1:]
  x = refined_body.split('|')
  URL ='https://' + x[128] + '-' + x[133] + '-' + x[134] + '.' + x[125] + '.' + x[136] + '/' + x[135] + '/' + x[139] + '/' + x[146] + '/' + x[148] + '.' + x[137]
  return URL


def fetch_m3u8(URL):
  splits = URL.split('/')
  HOST = splits[2]
  PATH = '/'+splits[3]+'/'+splits[4]+'/'+splits[5]+'/'+splits[6]
  HEADER = """:authority: """+HOST+"""
:method: GET
:path: """+PATH+"""
:scheme: https
accept: */*
accept-encoding: gzip, deflate, br
accept-language: en-US,en;q=0.9,bn;q=0.8
cache-control: no-cache
origin: https://kwik.cx
pragma: no-cache
referer: https://kwik.cx/e/O2lpweX6cBOQ
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: cross-site
user-agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36
host: """+HOST+"""
"""
  HEADER.replace('\n','\r\n')
  context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
  s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  s_sock = context.wrap_socket(s, server_hostname=HOST)
  s_sock.connect((HOST,443))
  s_sock.send(("GET "+PATH+" HTTP/1.1\r\n"+HEADER+"\r\n").encode('utf-8'))
  response = b''
  while True:
    data = s_sock.recv(4096)
    response = response + data
    if data[len(data)-5:] == b'0\r\n\r\n':
      break
    if len(data) < 1:
      break
  s_sock.close()
  data = extract_data_from_response(response)[1]
  try:
    data = gzip.decompress(data)
  except:
    pass
  return data


def fetch_ts_filename(url):
  return url.split('/')[::-1][0]

def create_file(name,data,path):
  import os
  if not os.path.exists(path):
    os.makedirs(path)
  with open(path+'/'+name, 'wb') as File:
    File.write(data)
    File.close()

def fetch_key_url(line):
  line = line.split(b'=')[2]
  return line[1:len(line)-1].decode()

def fetch_key(url):
  return fetch_ts(url)  

def fetch_ts(URL):
  tc = time.time()
  global u_time
  splits = URL.split('/')
  HOST = splits[2]
  PATH = '/'+splits[3]+'/'+splits[4]+'/'+splits[5]+'/'+splits[6]
  HEADER = """:authority: """+HOST+"""
:method: GET
:path: /stream/0000/8a708bf4d0fcfabf96f912072b4b37e2482c6cfb021e20122fcfa551e0390710/uwu.m3u8
:scheme: https
accept: */*
accept-encoding: gzip, deflate, br
accept-language: en-US,en;q=0.9,bn;q=0.8
cache-control: no-cache
origin: https://kwik.cx
pragma: no-cache
referer: https://kwik.cx/e/O2lpweX6cBOQ
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: cross-site
user-agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36
host: """+HOST+"""
"""
  HEADER.replace('\n','\r\n')
  context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
  s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  s_sock = context.wrap_socket(s, server_hostname=HOST)  
  s_sock.connect((HOST,443))  
  s_sock.send(("GET "+PATH+" HTTP/1.1\r\n"+HEADER+"\r\n").encode('utf-8'))
  response = b''
  tc = time.time() - tc
  while True:
    t = time.time()
    data = s_sock.recv(4096) 
    u_time = ((time.time() - t + tc) * 4096/len(data) + u_time)/2
    response = response + data    
    if response.find(b'Content-Length') != -1 and response.find(b'\r\n\r\n') != -1:
      body_length = int(response[response.find(b'Content-Length'):].split(b'\r\n')[0].split(b' ')[1])
      if body_length == len(response.split(b'\r\n\r\n')[1]):
        break
    if len(data) < 1:
      break     
  s_sock.close()
  return response.split(b'\r\n\r\n')[1]


def show_pogress(pogress, length=30):
  import sys  
  sys.stdout.flush()
  percentage = pogress*100
  pogress = int(pogress*length)
  sys.stdout.write('\rPogress: ['+'='*pogress+' '*(length-pogress)+'] '+str(int(percentage))+'%  ')


def show_ts_pogress(pogress, length=30):
  import sys
  global u_time
  sys.stdout.flush()
  try:
    speed = str(int(50/u_time)/100) + ' mbps'
  except:
    speed = '??? mbps'
  percentage = pogress*100
  pogress = int(pogress*length)
  sys.stdout.write('\rPogress: ['+'='*pogress+' '*(length-pogress)+'] '+str(int(percentage))+'%  '+speed)


def fetch_and_write_video_raw(url, path='/content'):
  import os
  kwik = url
  path = path
  data = fetch_m3u8(fetch_nextstream_url(kwik)).split(b'\n')
  if not os.path.exists(path):
    os.makedirs(path)
  with open(path+'/uwu.m3u8', 'wb') as m3u8:
    i = 0
    data_length = len(data)
    while i < data_length:
      line = data[i]     

      if line.find(b'#EXT-X-KEY') != -1:
        url = fetch_key_url(line)
        create_file('mon.key',fetch_key(url),path)
        line = line.replace(url.encode(), 'mon.key'.encode())
      if line.find(b'https') == 0:
        url = line[line.find(b'https'):].decode()
        file_name = fetch_ts_filename(url)      
        create_file(file_name,fetch_ts(url),path)
        line = line.replace(url.encode(), file_name.encode())   
      m3u8.write(line+b'\n')
      show_ts_pogress((i+1)/data_length)
      i = i+1
    m3u8.close()
    print('\n')

def convert_to_mp4(file_path,download_path):
  return subprocess.call(['ffmpeg', '-allowed_extensions', 'ALL', '-i', file_path, '-c', 'copy', '-bsf:a', 'aac_adtstoasc', download_path])



def fetch_video_data_as_json(id,session):
  conn = HTTPSConnection('animepahe.com')
  conn.request('GET', '/api?m=links&id='+id+'&session='+session+'&p=kwik')
  response = conn.getresponse()
  return json.loads(response.read().decode('utf-8'))

def downloader(x,path):
  try:
    fetch_and_write_video_raw(x[3],path)
  except:
    try:
      fetch_and_write_video_raw(x[3],path)
    except:
      try:
        fetch_and_write_video_raw(x[3],path)
      except:
        try:
          fetch_and_write_video_raw(x[3],path)
        except Exception as e:
          print('UNABLE TO FETCH EPISODE: '+x[1])
          print(e)

def clear_session():
  if os.path.exists(SESSION):
    os.remove(SESSION)
  
def clear():
  output.clear()

# Status

In [ ]:
TEMP_DIR = '/content/animpahe_downloader_cache'
ANIME_NAME = ANIME_NAME.replace(' ','_')
SESSION = DOWNLOAD_DIR+'/'+ANIME_NAME+'/session.data'
if not os.path.exists(SESSION):
  print('Creating new session'+SESSION)
  file_name = SESSION.split('/')[::-1][0]
  path = SESSION.replace('/'+file_name,'')
  if not os.path.exists(path):
    os.makedirs(path)
  open(SESSION,'w+').close()
  
session = []
session_reader = open(SESSION,'r')
sessions = session_reader.read()
if sessions != '':
  session = sessions.split('\n')
session_reader.close()
quality = ''

if not QUALITY == 0:
  quality = str(QUALITY)

threads = []

print('FETCHING FROM: '+URL)
id, data = fetch_session(URL)

print('FETCHING EPISODE LIST:')
data, qualities= fetch_kwik_data(id,data)

if quality == '':
  quality = get_highest_quality(qualities)

print("TARGET QUALITY: "+quality+'p')



for x in data:
  if x[0] == quality and not x[3] in session:
    path = DOWNLOAD_DIR+'/'+ANIME_NAME+'/'+ANIME_NAME+'_'+x[2]+'_'+x[3].split('/')[::-1][0]+'_e'+x[1]+'_'+x[0]+'p.mp4'
    temp_path = TEMP_DIR+'/'+ANIME_NAME+x[2]+'/e'+x[1]+'_'+x[0]+'p'+x[3].split('/')[::-1][0]
    print('DOWNLOADING EPISODE: ' + x[1])
    print('PATH: '+path)
    downloader(x, temp_path)
    convert_to_mp4(temp_path+'/uwu.m3u8',path) 
    with open(SESSION,'a') as session_writer:
      session_writer.write(x[3]+'\n')
      session_writer.close()
    clear()

print('Download complete')

# Clear session

Clear the session if you want to redownload the series.

In [ ]:
clear_session()